# TMIP-EMAT Running Core Model Experiments on Multiple Computers

Experiments designed for sensitivity testing and meta-model development are run using the core model (as opposed to a meta-model). These experiments are completely independent and thus can be run in parallel if multiple computers are available. In this situation, an initial process is needed to create the TMIP-EMAT database, populate it with the scope, and design the experiments. At this point, jupyter notebooks on separate computers can connect to the same database to access experiment designs and to import metrics from the core model run.

The notebook cells below demonstrate the initial configuration that is only run **once**, that involves: 
1. Creating a TMIP-EMAT database
2. Storing the scope in the database
3. Designing a set of experiments

The following section demonstrates the execution steps that are run **on each modeling computer**, these involve:

1. **Connecting** to the TMIP-EMAT database
2. **Reading** the scope from the database
3. Running a subset of the experiments

In [2]:
import emat
emat.__version__

'0.1.4'

In [ ]:
import pandas, numpy, os
from emat.util.loggers import log_to_stderr
log = log_to_stderr(level=20)
scope = emat.Scope('GBNRTC_scope.yaml') # scope file is local to the Master

## Create SQLite DB file and record scope

The database file needs to be stored in a location accessible by all modeling systems

In [ ]:
db = emat.SQLiteDB("emat_db/gbnrtc_multi.db",initialize=True)
scope = emat.Scope("gbnrtc_scope.yaml")

## Load Model

In [ ]:
from emat.model import GBNRTCModel
g = GBNRTCModel(
    configuration='gbnrtc_model_config.yaml',
    scope=scope,
    db=db,
)
g

### Design experiments

Only done by Master notebook

In [ ]:
design = g.design_experiments(scope=scope,
                              n_samples_per_factor=10, 
                              sampler='lhs',
                              design_name='test_lhs')
design.head()

# Run subset of experiments on each computer

Results will be stored in the database associated with the model

***The following cells are to be run in an independent notebook***

In [ ]:
import emat
emat.__version__


In [ ]:
import pandas, numpy, os
from emat.util.loggers import log_to_stderr
log = log_to_stderr(level=20)
#scope = emat.Scope('GBNRTC_scope.yaml') # Scope will be read from the database

## Connect to existing SQLite DB file and read scope

File created by Master notebook (step 1.1 must be complete by the Master notebook)

In [ ]:
db = emat.SQLiteDB("emat_db/gbnrtc.db",initialize=False)
scope = db.read_scope('GBNRTC')

## Load Model

In [ ]:
from emat.model import GBNRTCModel
g = GBNRTCModel(
    configuration='gbnrtc_model_config.yaml',
    scope=scope,
    db=db,
)
g

### Run subset of experiments

Run set of experiments (1-35)

Results will be stored in the database associated with the model

In [ ]:
design_subset = g.read_experiments(design_name='gbnrtc_lhs').iloc[0:35]
lhs_results = g.run_experiments(design_subset)